# Imports

In [1]:
%run ~/REGE/utils.py

# Runs

In [2]:
files = os.listdir('/home/shafi.z/REGE/data')
data_name = ['citeseer', 'cora', 'coraml', 'polblogs']
attack_type = 'grad'
num_runs = 1
res_dict = {}

variance_hyp = {
    'citeseer' : {'rege(d)' : 10, 
                  'rege(m)' : 10}, 
    'cora' : {'rege(d)' : 1, 
                  'rege(m)' : 5},
    'coraml' : {'rege(d)' : 1, 
                  'rege(m)' : 5},
    'polblogs' : {'rege(d)' : 1, 
                  'rege(m)' : 1},
}

for d in data_name:
    
    data_files = ['/home/shafi.z/REGE/data/' + d + '_' + str(ptb) + '.pkl' for ptb in [0.01, 0.1]]
    
    for graph_name in data_files:
        
        try:
            graph_name = graph_name.replace("/home/shafi.z/REGE/data/", "")
            graph_name = graph_name.replace(".pkl", "")
            print ("============================================")
            print (graph_name) 
            print ("============================================")
            
            res_dict[graph_name] = {}
    
            pre_attack_dataset = d + '.npz'
    
            graph_dict = get_graph(graph_name = graph_name,
                                   attack_type = attack_type, 
                                   pre_attack_dataset = pre_attack_dataset, 
                                   data_dir = '/home/shafi.z/REGE/data/', 
                                   ensure_connected = False)
    
            variance_dict = get_variances(graph = graph_dict['graph'],
                                                      dist_mat = None,
                                                      sample_dict = None,
                                                      sample_count = 50,
                                                      eigen_step = 25,
                                                      plot = False, 
                                                      types = ['conformal', 'eigen'], 
                                                      graph_dict = graph_dict)
            
            features = scipy.sparse.csr_array(graph_dict['features'].cpu().numpy())
            labels = graph_dict['labels'].cpu()
            adj = graph_dict['adj']
            idx_train = graph_dict['idx_train']
            idx_test = graph_dict['idx_test']
    
            device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    
            for _ in range(num_runs):
    
                surrogate = GCN(nfeat=features.shape[1], nclass=labels.max().item()+1, nhid=16,
                                with_relu=False, device=device)
                surrogate = surrogate.to(device)
                surrogate.fit(graph_dict['features'], adj, labels, idx_train)
                plain_gcn_test = surrogate.test(idx_test)
    
                try:
                    res_dict[graph_name]['gcn'].append(plain_gcn_test)
                except: 
                    res_dict[graph_name]['gcn'] = [plain_gcn_test]
                
    
                eigen_sampled_var_model, eigen_sampled_var_test, r = train_eigen_sampled_variance_model(graph_dict = graph_dict,
                                                                                                    sample_dict = None,
                                                                                                    curvature_dict = None,
                                                                                                    variance = variance_dict['binary_deviation'],
                                                                                                    variance_hyp = variance_hyp[d]['rege(d)'],
                                                                                                    nhid = 16,
                                                                                                    lr = 1e-4,
                                                                                                    eigen_count = 5,
                                                                                                    train_iters = 100,
                                                                                                    adjacency = None,
                                                                                                    patience = 25)
                try:
                    res_dict[graph_name]['rege(d)'].append(eigen_sampled_var_test)
                except: 
                    res_dict[graph_name]['rege(d)'] = [eigen_sampled_var_test]
                    
    
                eigen_sampled_var_model, eigen_sampled_var_test, r = train_eigen_sampled_variance_model(graph_dict = graph_dict,
                                                                                                    sample_dict = None,
                                                                                                    curvature_dict = None,
                                                                                                    variance = variance_dict['conformal_radii'],
                                                                                                    variance_hyp = variance_hyp[d]['rege(m)'],
                                                                                                    nhid = 16,
                                                                                                    lr = 1e-4,
                                                                                                    eigen_count = 5,
                                                                                                    train_iters = 100,
                                                                                                    adjacency = None,
                                                                                                    patience = 25)
                try:
                    res_dict[graph_name]['rege(m)'].append(eigen_sampled_var_test)
                except: 
                    res_dict[graph_name]['rege(m)'] = [eigen_sampled_var_test]


        except Exception as e: 
            print (e)
            continue
    

citeseer_0.01
Test set results: loss= 2.0820 accuracy= 0.6713
Epoch :  2840  | Loss :  0.44641879200935364
Early Stopping at Epoch 2840
Teacher Acc :  0.671320754716981
Student Acc :  0.6852830188679245


100%|██████████| 133/133 [00:36<00:00,  3.64it/s]
/home/shafi.z/REGE/utils.py:915: RuntimeWarning: divide by zero encountered in divide
  eig_variance = 1 / np.mean(eig_variance, axis = 0)
/home/shafi.z/REGE/utils.py:916: RuntimeWarning: invalid value encountered in divide
  eig_variance = (eig_variance - np.min(eig_variance)) / np.ptp(eig_variance)


Test set results: loss= 2.0437 accuracy= 0.6721


  0%|          | 0/662 [00:00<?, ?it/s]

Epoch 0, training loss: 18.610233306884766
Epoch 10, training loss: 16.424623489379883
Epoch 20, training loss: 18.770038604736328
Epoch 30, training loss: 18.006561279296875
Epoch 40, training loss: 17.190099716186523
Epoch 50, training loss: 16.603673934936523
Epoch 60, training loss: 16.860515594482422
Epoch 70, training loss: 15.344975471496582
Epoch 80, training loss: 15.847172737121582
Epoch 90, training loss: 19.187702178955078


...

Epoch 0, training loss: 0.019925180822610855
Epoch 10, training loss: 0.019707821309566498
Epoch 20, training loss: 0.01676720380783081
Epoch 30, training loss: 0.023471016436815262
Epoch 40, training loss: 0.01478633377701044
Epoch 50, training loss: 0.01858297549188137
Epoch 60, training loss: 0.018968958407640457
Epoch 70, training loss: 0.021878715604543686
Epoch 80, training loss: 0.014719393104314804
Epoch 90, training loss: 0.017175152897834778
Component :  385 Test Accuracy :  0.8557046979865771
Component :  385 Val Accuracy :  0.8120805369127516
Epoch 0, training loss: 0.02011985518038273


 26%|██▋       | 78/297 [00:14<00:41,  5.23it/s]

Epoch 10, training loss: 0.01598016545176506
Epoch 20, training loss: 0.02104843221604824
Epoch 30, training loss: 0.010141052305698395
Epoch 40, training loss: 0.01827867329120636
Epoch 50, training loss: 0.018428422510623932
Epoch 60, training loss: 0.024785635992884636
Epoch 70, training loss: 0.01585467718541622
Epoch 80, training loss: 0.017441576346755028
Epoch 90, training loss: 0.016816265881061554
Component :  390 Test Accuracy :  0.8557046979865771
Component :  390 Val Accuracy :  0.8120805369127516
Epoch 0, training loss: 0.02028532512485981
Epoch 10, training loss: 0.011174676939845085


 26%|██▋       | 78/297 [00:15<00:42,  5.19it/s]

Epoch 20, training loss: 0.017246471717953682
Epoch 30, training loss: 0.037583690136671066
Epoch 40, training loss: 0.01473259087651968
Epoch 50, training loss: 0.017859021201729774
Epoch 60, training loss: 0.027866041287779808
Epoch 70, training loss: 0.020356914028525352
Epoch 80, training loss: 0.026077236980199814
Epoch 90, training loss: 0.017846133559942245
Component :  395 Test Accuracy :  0.8557046979865771
Component :  395 Val Accuracy :  0.8120805369127516
Early Stopping at 270 components, test accuracy : 0.8582214765100671


In [3]:
res_dict

{'citeseer_0.01': {'gcn': [0.6720754716981132],
  'rege(d)': [0.7215094339622641],
  'rege(m)': [0.7109433962264151]},
 'citeseer_0.1': {'gcn': [0.6358490566037736],
  'rege(d)': [0.7064150943396226],
  'rege(m)': [0.7026415094339622]},
 'cora_0.01': {'gcn': [0.7937240424550069],
  'rege(d)': [0.8366405168435626],
  'rege(m)': [0.843101061375173]},
 'cora_0.1': {'gcn': [0.7914167051222889],
  'rege(d)': [0.8061836640516843],
  'rege(m)': [0.8154130133825566]},
 'coraml_0.01': {'gcn': [0.8284641068447413],
  'rege(d)': [0.8580968280467446],
  'rege(m)': [0.8568447412353923]},
 'coraml_0.1': {'gcn': [0.8096828046744574],
  'rege(d)': [0.835559265442404],
  'rege(m)': [0.8368113522537562]},
 'polblogs_0.01': {'gcn': [0.8783557046979865],
  'rege(d)': [0.8901006711409396],
  'rege(m)': [0.8766778523489933]},
 'polblogs_0.1': {'gcn': [0.799496644295302],
  'rege(d)': [0.8758389261744967],
  'rege(m)': [0.8582214765100671]}}